Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

# 1. Подготовка данных

Для начала необходимо симпортировать библиотеки

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from catboost import CatBoostRegressor
import lightgbm as lgbm
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
import warnings
warnings.filterwarnings("ignore")

После импорта считываю датасет

In [2]:
df = pd.read_csv('/datasets/autos.csv')

Далее просматриваю датасет на наличие пропусков

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
DateCrawled          354369 non-null object
Price                354369 non-null int64
VehicleType          316879 non-null object
RegistrationYear     354369 non-null int64
Gearbox              334536 non-null object
Power                354369 non-null int64
Model                334664 non-null object
Kilometer            354369 non-null int64
RegistrationMonth    354369 non-null int64
FuelType             321474 non-null object
Brand                354369 non-null object
NotRepaired          283215 non-null object
DateCreated          354369 non-null object
NumberOfPictures     354369 non-null int64
PostalCode           354369 non-null int64
LastSeen             354369 non-null object
dtypes: int64(7), object(9)
memory usage: 43.3+ MB


В датасете обнаружены следующие проблемы:
- Есть пропуски в признаках: VehicleType, Gearbox, Model, FuelType, NotRepaired
- В названиях признаков есь заглавные буквы, что для меня не удобно
- Признаки, обозначающие даты будет необходимо привести к формау datetime

Смотрю как выглядят данные

In [4]:
df.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


Еще раз смотрю какие признаки содержат пропуски

In [5]:
df.isna().sum()

DateCrawled              0
Price                    0
VehicleType          37490
RegistrationYear         0
Gearbox              19833
Power                    0
Model                19705
Kilometer                0
RegistrationMonth        0
FuelType             32895
Brand                    0
NotRepaired          71154
DateCreated              0
NumberOfPictures         0
PostalCode               0
LastSeen                 0
dtype: int64

Так как пропуски содержатся только в категориалных признаках я решил удалить эти строки, так как заполнит их возможности нет

In [6]:
df.dropna(inplace=True)

Проверяю корректность удаления

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 245814 entries, 3 to 354367
Data columns (total 16 columns):
DateCrawled          245814 non-null object
Price                245814 non-null int64
VehicleType          245814 non-null object
RegistrationYear     245814 non-null int64
Gearbox              245814 non-null object
Power                245814 non-null int64
Model                245814 non-null object
Kilometer            245814 non-null int64
RegistrationMonth    245814 non-null int64
FuelType             245814 non-null object
Brand                245814 non-null object
NotRepaired          245814 non-null object
DateCreated          245814 non-null object
NumberOfPictures     245814 non-null int64
PostalCode           245814 non-null int64
LastSeen             245814 non-null object
dtypes: int64(7), object(9)
memory usage: 31.9+ MB


Далее строю график для поиска ошибок в данных

In [8]:
#sns.pairplot(df)

Удаляю признак с количеством изображений, так как на всех записях он равен 0

In [9]:
del df['NumberOfPictures']

проверяю, нет ли ошибок в зполнении месяцев

In [10]:
df['RegistrationMonth'].value_counts()

3     26527
6     23390
4     21950
5     21582
7     20262
10    19685
9     18167
12    18112
11    18070
1     17393
8     16907
2     16238
0      7531
Name: RegistrationMonth, dtype: int64

Так как нулевого месяца не существует - удаляю записи с нулевым месяцем

In [11]:
df = df[df['RegistrationMonth'] > 0]

Из графика видно, что некоторые автомобили имеют мощность гораздо больше 1000 л.с., что не совсем соответствует действительности. Поэтому решаю удалить все автомобили с мощностью более 1000 л.с.

In [12]:
df = df[df['Power'] < 1000]

Оцениваю оставшийся датасет по объему

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 238161 entries, 3 to 354367
Data columns (total 15 columns):
DateCrawled          238161 non-null object
Price                238161 non-null int64
VehicleType          238161 non-null object
RegistrationYear     238161 non-null int64
Gearbox              238161 non-null object
Power                238161 non-null int64
Model                238161 non-null object
Kilometer            238161 non-null int64
RegistrationMonth    238161 non-null int64
FuelType             238161 non-null object
Brand                238161 non-null object
NotRepaired          238161 non-null object
DateCreated          238161 non-null object
PostalCode           238161 non-null int64
LastSeen             238161 non-null object
dtypes: int64(6), object(9)
memory usage: 29.1+ MB


Привожу названия признаков к нижнему регистру

In [14]:
df.columns = map(str.lower, df.columns)

Обновляю индексы

In [15]:
df = df.reset_index(drop=True)

Привожу даты к формату datetime

In [16]:
df['datecrawled'] = pd.to_datetime(df['datecrawled'], format='%Y-%m-%d %H:%M:%S')
df['datecreated'] = pd.to_datetime(df['datecreated'], format='%Y-%m-%d %H:%M:%S')
df['lastseen'] = pd.to_datetime(df['lastseen'], format='%Y-%m-%d %H:%M:%S')

Проверяю успешность всех выполненных действий:)

In [17]:
df.head()

,datecrawled,price,vehicletype,registrationyear,gearbox,power,model,kilometer,registrationmonth,fueltype,brand,notrepaired,datecreated,postalcode,lastseen
0,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17,91074,2016-03-17 17:40:17
1,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31,60437,2016-04-06 10:17:21
2,2016-04-04 17:36:23,650,sedan,1995,manual,102,3er,150000,10,petrol,bmw,yes,2016-04-04,33775,2016-04-06 19:17:07
3,2016-04-01 20:48:51,2200,convertible,2004,manual,109,2_reihe,150000,8,petrol,peugeot,no,2016-04-01,67112,2016-04-05 18:18:39
4,2016-03-21 18:54:38,0,sedan,1980,manual,50,other,40000,7,petrol,volkswagen,no,2016-03-21,19348,2016-03-25 16:47:58


Удаляю эти признаки, выявленно опытным путем что после их удаления ошибка уменьшилась

In [18]:
del df['datecrawled']
del df['lastseen']
del df['datecreated']
del df['postalcode']

Разделяю датасет на признаковое пространство и целевой признак

In [19]:
X = df.drop('price', axis=1)
y = df['price'] 

Разбиваю датасет на обучающюю и тестовую выборки в соотношении 80-20

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12345)

In [21]:
X_train_lgbm = X_train
X_test_lgbm = X_test

### Выводы

# 2. Обучение моделей

В данном разделе будут рассмотрены три модели:
- CatBoostRegressor
- LightGBM
- LinearRegression без настройки

Для начала перечисляю список категориальных переменных для CatBoostRegressor

In [22]:
categ_features = [
    #'datecrawled',
    'vehicletype',
    'gearbox',
    'model',
    'fueltype',
    'brand',
    'notrepaired'
    #'datecreated',
    #'lastseen'
]

Далее разбиваю обучение и предсказание на разные ячейки для выяснения времени каждого процесса. Для CatBoostRegressor выбраны параметры loss_function='RMSE', iterations=150, depth=10. Поискать оптимальные параметры не особо получилось, эти параметры взяты на глаз, но при этом показывают неплохие результаты

In [23]:
%%time
model_cbr = CatBoostRegressor(loss_function='RMSE', iterations=150, depth=10)
model_cbr.fit(X_train, y_train, cat_features=categ_features, verbose=10)


0:	learn: 4629.4219772	total: 467ms	remaining: 1m 9s
10:	learn: 3773.6492863	total: 4.46s	remaining: 56.4s
20:	learn: 3188.8143907	total: 8.26s	remaining: 50.7s
30:	learn: 2791.6560932	total: 12.2s	remaining: 46.7s
40:	learn: 2497.3309689	total: 16s	remaining: 42.7s
50:	learn: 2292.5192617	total: 19.9s	remaining: 38.7s
60:	learn: 2149.4017947	total: 23.8s	remaining: 34.8s
70:	learn: 2048.7752236	total: 27.6s	remaining: 30.7s
80:	learn: 1980.3686908	total: 31.4s	remaining: 26.8s
90:	learn: 1933.2720069	total: 35.2s	remaining: 22.8s
100:	learn: 1888.5528019	total: 38.9s	remaining: 18.9s
110:	learn: 1856.9911004	total: 42.8s	remaining: 15s
120:	learn: 1831.3462429	total: 46.6s	remaining: 11.2s
130:	learn: 1813.7751545	total: 50.6s	remaining: 7.34s
140:	learn: 1798.7782535	total: 54.3s	remaining: 3.46s
149:	learn: 1787.0578106	total: 57.8s	remaining: 0us
CPU times: user 52.9 s, sys: 5.82 s, total: 58.8 s
Wall time: 1min


Время на обучение модели почти 60 секунд, немного долго, но это время можно отрегулировать параметром iterations (чем меньше - тем быстрее)

Далее смотрю время на предсказании и точность

In [24]:
%%time
pred = model_cbr.predict(X_test)
print("RMSE=", mean_squared_error(y_test, pred)**0.5)

RMSE= 1815.6985400982269
CPU times: user 184 ms, sys: 13.3 ms, total: 197 ms
Wall time: 170 ms


Предсказывает достаточно быстро, ошибка вполне устраивает

Далее перечисляю гиперпараметры для LightGBM, параметры тоже взяты весьма случайно, есть проблема с пониманием того, какие значения параметров ожидает алгоритм

In [25]:
params_lgbm = {
    "num_boost_round":25,
    "max_depth" : 10,
    "num_leaves" : 150,
    'learning_rate' : 0.1,
    'boosting_type' : 'gbdt'
}

Так как в LightGBM и LinearRegression нет встроенного кодировщика категориальных признаков, необходимо закодировать эти признаки. Выбран LabelEncoder, просто он мне больше нравится

Так же кодируем для теста

In [27]:
# le.fit(X_test['vehicletype'])
# X_test['vehicletype'] = le.transform(X_test['vehicletype'])

# le.fit(X_test['gearbox'])
# X_test['gearbox'] = le.transform(X_test['gearbox'])

# le.fit(X_test['model'])
# X_test['model'] = le.transform(X_test['model'])

# le.fit(X_test['fueltype'])
# X_test['fueltype'] = le.transform(X_test['fueltype'])

# le.fit(X_test['brand'])
# X_test['brand'] = le.transform(X_test['brand'])

# le.fit(X_test['notrepaired'])
# X_test['notrepaired'] = le.transform(X_test['notrepaired'])

Для того, чтобы LightGBM мог работать с категориальными признаками, необходимо приветси переменные к типу category 

In [28]:
X_train_lgbm['vehicletype'] = X_train_lgbm['vehicletype'].astype('category')
X_train_lgbm['gearbox'] = X_train_lgbm['gearbox'].astype('category')
X_train_lgbm['model'] = X_train_lgbm['model'].astype('category')
X_train_lgbm['fueltype'] = X_train_lgbm['fueltype'].astype('category')
X_train_lgbm['brand'] = X_train_lgbm['brand'].astype('category')
X_train_lgbm['notrepaired'] = X_train_lgbm['notrepaired'].astype('category')

In [ ]:
X_test_lgbm['vehicletype'] = X_test_lgbm['vehicletype'].astype('category')
X_test_lgbm['gearbox'] = X_test_lgbm['gearbox'].astype('category')
X_test_lgbm['model'] = X_test_lgbm['model'].astype('category')
X_test_lgbm['fueltype'] = X_test_lgbm['fueltype'].astype('category')
X_test_lgbm['brand'] = X_test_lgbm['brand'].astype('category')
X_test_lgbm['notrepaired'] = X_test_lgbm['notrepaired'].astype('category')

Далее при помощи GridSearchCV ищутся оптимальные значения для параметров num_leaves и max_depth (дал небольшой диапазон значений, так как при большем диапазоне параметры не нашлись за сутки)

In [30]:
%%time
param_grid={'num_leaves': [num_leaves for num_leaves in range (150, 155)],
           'max_depth': [max_depth for max_depth in range (10, 12)]}
gs_rf = GridSearchCV(lgbm.sklearn.LGBMRegressor(), param_grid=param_grid)
gs_rf.fit(X_train_lgbm, y_train)
gs_rf.best_params_
bp = gs_rf.best_params_

CPU times: user 7min 24s, sys: 3.21 s, total: 7min 27s
Wall time: 7min 32s


Далее обучается модель LGBMRegressor, значения параметров для него были указаны выше

Далее в модель передаются найденные параметры и модель обучаеся

In [32]:
%%time
model_lgbm = lgbm.sklearn.LGBMRegressor(
    num_leaves=bp["num_leaves"], 
    n_estimators=params_lgbm["num_boost_round"], 
    max_depth=bp["max_depth"],
    learning_rate=params_lgbm["learning_rate"]
)
model_lgbm.fit(X_train, y_train)

CPU times: user 4.63 s, sys: 29.4 ms, total: 4.66 s
Wall time: 4.69 s


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=10,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=25, n_jobs=-1, num_leaves=150, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

Обучается модель гораздо быстрее чем CatBoostRegressor

In [33]:
%%time
pred_lgbm = model_lgbm.predict(X_test)
print("RMSE=", mean_squared_error(y_test, pred_lgbm)**0.5)

RMSE= 1762.3330491485274
CPU times: user 282 ms, sys: 4.36 ms, total: 286 ms
Wall time: 295 ms


Скорость предсказания ниже, но ошибка сопоставима с CatBoostRegressor

Далее LinearRegression. Эту модель взял из любопытства, было интересно, что покажет модель без тюнинга

Для начала необходимо закодировать переменные

In [40]:
le = preprocessing.LabelEncoder()
# Так не работает
# le.fit(X_train[['vehicletype', 'gearbox', 'model', 'fueltype', 'brand', 'notrepaired']])
# X_train[['vehicletype', 'gearbox', 'model', 'fueltype', 'brand', 'notrepaired']] = le.transform(X_train[['vehicletype', 'gearbox', 'model', 'fueltype', 'brand', 'notrepaired']])
# X_test[['vehicletype', 'gearbox', 'model', 'fueltype', 'brand', 'notrepaired']] = le.transform(X_test[['vehicletype', 'gearbox', 'model', 'fueltype', 'brand', 'notrepaired']])

le.fit(X_train['vehicletype'])
X_train['vehicletype'] = le.transform(X_train['vehicletype'])
X_test['vehicletype'] = le.transform(X_test['vehicletype'])

le.fit(X_train['gearbox'])
X_train['gearbox'] = le.transform(X_train['gearbox'])
X_test['gearbox'] = le.transform(X_test['gearbox'])

le.fit(X_train['model'])
X_train['model'] = le.transform(X_train['model'])
X_test['model'] = le.transform(X_test['model'])

le.fit(X_train['fueltype'])
X_train['fueltype'] = le.transform(X_train['fueltype'])
X_test['fueltype'] = le.transform(X_test['fueltype'])

le.fit(X_train['brand'])
X_train['brand'] = le.transform(X_train['brand'])
X_test['brand'] = le.transform(X_test['brand'])

le.fit(X_train['notrepaired'])
X_train['notrepaired'] = le.transform(X_train['notrepaired'])
X_test['notrepaired'] = le.transform(X_test['notrepaired'])

In [41]:
%%time
model_lr = LinearRegression()
model_lr.fit(X_train, y_train)

CPU times: user 65.9 ms, sys: 17 ms, total: 82.9 ms
Wall time: 56 ms


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

Скорость обучения очень высокая

In [42]:
%%time
pred_lr = model_lr.predict(X_test)
print("RMSE=", mean_squared_error(y_test, pred_lr)**0.5)

RMSE= 2984.36894208168
CPU times: user 8.2 ms, sys: 12.4 ms, total: 20.6 ms
Wall time: 6.72 ms


Скорость предсказания тоже очень высокая, но ошибка почти в 1.5 раза выше, чем у CatBoostRegressor

### Выводы
- Обучено 3 модели
- Для каждой модели вычислены время обучения, время предсказания и найдена RMSE

# 3. Анализ моделей

Ниже приведена сводная <a id="table"></a>таблица для трех моделей

Модель|Время на обучение|Время на предсказание|Ошибка (RMSE)
------|-----------------|---------------------|-------------
CatBoostRegressor|60 s.|190 ms.|1815
LightGBM|5 s.|295 ms.|1762
LinearRegression|49 ms.|5 ms.|2984

Выводы по таблице:
- Самая долгая модель - CatBoostRegressor
- Самая быстрая модель - LinearRegression
- Самая точная модель - CatBoostRegressor
- Самое лучшее соотношения времени обучения и точности - **LightGBM**

## Выводы
- Исходный датасет был предобработан (удалены пропуски, ненужные признаки, переименованы признаки)
- Предобработанный датасет разделен на обучающую и тестовую выборки
- Обучено три модели регрессии (CatBoostRegression, LightgbM, LiearRegression)
- Найдено время обучения и RMSE (см. [таблицу](#table))
- Выбрана оптимальная модель для заказчика (LightGBM c параметрами:"num_boost_round":25, "max_depth" : 11, "num_leaves" : 151, 'learning_rate' : 0.1, 'boosting_type' : 'gbdt' )


Для текщей задачи важнее всего точность и время предсказания, так как модель обучается один раз (при найденных гиперпараметрах). Так что для заказчика время обучения роли не играет, но так как время предсказания у всех моделей очен низкое (меньше секунды) то соотвественно модель выбирается только на основе ошибки. В данном примере такой моделью оказалась LightGBM с перечисленными выше параметрами